In [ ]:
import oandapyV20
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import MySQLdb
from datetime import datetime
from dateutil import parser
import json
 
with open('config.json', 'r') as f:
    config = json.load(f) 
    
access_token = config['liveToken']
client = oandapyV20.API(access_token=access_token,environment='live')
  
_from = "2020-07-24T00:00:00Z"
_to = "2021-02-03T05:00:00Z"

currencies = [  
               'AUD_USD',
               'CAD_JPY',
               'EUR_CHF',
               'EUR_GBP',
               'EUR_JPY', 
               'EUR_USD',
               'GBP_CHF',
               'GBP_JPY',
               'NZD_USD',
               'USD_CAD',
               'USD_CHF',
               'USD_JPY',
               'EUR_AUD',
               'GBP_USD',
]

instruments = currencies

granularities = ['M5', 'M15', 'M30', 'H1', 'H3', 'H4', 'D', 'W']


conn = MySQLdb.connect(host=config['dbHost'], user=config['dbUser'],passwd=config['dbPass'],db='forexV20')
duplicates = []
for granularity in granularities:
    print(granularity)
    for instrument in instruments:
        print(instrument)
        table = instrument.lower()+'_'+granularity.lower()
        x = conn.cursor()

        params = {
            "from": _from,
            "count":4900,
            "to": _to,
            "granularity": granularity,
            "price": 'BAM'

        }
        for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
            client.request(r)
            candles = r.response.get('candles')

            for candle in candles:
                if not candle['complete']:
                    print(candle)
                    continue
                try:
                    x.execute("INSERT INTO "+table+" VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
                                  (parser.parse(candle['time'],ignoretz=True),
                                   candle['mid']['o'],candle['mid']['h'],candle['mid']['l'],candle['mid']['c'],
                                   candle['bid']['o'],candle['bid']['h'],candle['bid']['l'],candle['bid']['c'],
                                   candle['ask']['o'],candle['ask']['h'],candle['ask']['l'],candle['ask']['c'],
                                   candle['volume']))
                except Exception as e:
                    if 'Duplicate entry' in str(e):
                        duplicates.append(candle)
                    else:
                        print(str(e))
                        print(candle)
        conn.commit()
print('Num Duplicates: ',len(duplicates))
conn.close()